In [ ]:
## A `Dim` is just an integer, but it's used to represent the dimensions of a 2-dimensional tensor.
## Use of the `Dim` class allows us to track matrix dimensions to statically check for shape errors.

from modugant.matrix.dim import Dim
from modugant.matrix.matrix import Matrix

## A `Dim` instance is only different from an integer in static analysis
## it is identical to an int at run-time
## create Dim types for the dimensions of the matrices
D = Dim[3] ## Literal 3 type, but tagged as its own class
T = Dim[5] ## Literal 5 type, but tagged as its own class

d = Dim[3](3) ## a literal 3 instance, no different from int 3 at run-time
t = Dim[5](5) ## a literal 5 instance, no different from int 5 at run-time

## A `Matrix` instance is only different from a strictly 2-dimensional tensor in static analysis
## it is identical to a 2-dimensional tensor at run-time
## create Matrix types for the input and output matrices
ThreeByFive = Matrix[Dim[3], Dim[5]] ## 3x5 Tensor type

def transform(data: Matrix[Dim[int], D], weight: Matrix[D, T]) -> Matrix[Dim[int], T]:
    result = data @ weight ## correctly infered as a Matrix[int, T]
    try:
        bad_mult = data * weight ## correctly flagged as a static error
        bad_mat_mul = data @ data ## correctly flagged as a static error
    except TypeError as e:
        print(e)
    return result

## N generic to track row dimension as well
def transform_gen[N: Dim[int]](data: Matrix[N, D], weight: Matrix[D, T]) -> Matrix[N, T]:
    return data @ weight ## correctly infered as a Matrix[N, T]

## The `Dim` is not necessary in this case
def transform_no_dim[N: int](data: Matrix[N, D], weight: Matrix[D, T]) -> Matrix[N, T]:
    return data @ weight ## correctly infered as a Matrix[N, T]

In [3]:
## `Zero` and `One` are helper types that represent `Dim[0](0)`` and `Dim[1](1)`` respectively

from modugant.matrix.dim import One, Zero

## Generate a `Zero` or `One` type with static methods of `Dim`
zero: Zero = Dim.zero()
one: One = Dim.one()

## many Tensor operations have been retyped in the inheriting Matrix class to maintain shape information where knowable

def sum[R: int, C: int](matrix: Matrix[R, C]) -> Matrix[R, One]:
    return matrix.sum(dim = 1, keepdim = True) ## correctly infered dimensions

def funky_sum[R: int, C: int](matrix: Matrix[R, C]) -> Matrix[R, One]:
    return matrix.t().sum(dim = 0, keepdim = True).T ## correctly infered dimensions through sum and transposes

In [ ]:
## Create a `Matrix` from a `Tensor` by specifying its dimensions and the `load` static method

from torch import tensor

shape = (Dim[3](3), Dim[5](5))

t_data = tensor([
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13, 14, 15]
])

## keeping the `Matrix` type with static dimensions does tend to lead to more verbose code
## The `shape` is knowable, so the `Dim` types are not necessary at run-time, but used for static dimensions
m_data = Matrix.load(t_data, shape)

In [ ]:
## Use special replacement functions to generate `Matrix` types

## The functions in ...ops.py generally follow some subset of the tensor version signatures

from modugant.matrix.ops import cat, ones, rand, randn, zeros

## Generate a `Matrix` of zeros
zeros_m = zeros(shape)

## Generate a `Matrix` of ones
ones_m = ones(shape)

## Generate a `Matrix` of random normal values
randn_m = randn(shape)

## Generate a `Matrix` of random values
rand_m = rand(shape)

## `result` dimensions are correctly inferred as `[1, 3]` through the operations
result = ((zeros_m + ones_m * randn_m) @ rand_m.T).mean(dim = 0, keepdim = True)

## cat requires a resulting dimension to be specified
try:
    cat_m = cat((zeros_m, ones_m, randn_m), dim = 1, shape = (10, 15)) ## shape does not conform to matrix arguments
except TypeError as e:
    pass

## row dim must be 3, col will be checked at run-time
cat_m_c = cat((zeros_m, ones_m, randn_m), dim = 1, shape = (Dim[3](3), 15))
 ## col dim must be 5, row will be checked at run-time
cat_m_r = cat((zeros_m, ones_m, randn_m), dim = 0, shape = (10, Dim[5](5)))

In [ ]:
## The `Index` class helps us maintain static dimension information through indexing as well
## The `Index` class is only different from an integer in static analysis

from modugant.matrix.index import Index

## An `Index` instance is only different from an List[int] in static analysis
## it is identical to a List[int] at run-time
## Create an `Index` type for the row dimension
idx = Index.load([0, 2], Dim[2](2))

matrix = Matrix.load(t_data, shape)

## Use Ellipsis to index-all for a `Matrix` type.
## This helps us keep `Matrix` static dimension information
sub_matrix = matrix[idx, ...] @ rand_m.T

## Matrices must be indexed with Sequences in-order not to drop down to 1 or 0 dimensions
## Index a single row/column
try:
    cell = matrix[0, 0] ## integer indexing drops a dimension
except TypeError as e:
    print(e)

cell = matrix[[0], [0]] ## subset without dropping a dimension, however dimensions are inferred as [Any, Any]

## Maintain static dimensions by using `Index` types
## Use `Index.at` for convenience instead of `Index.load([0], Dim[1](1))` or `Index.load([0], Dim.one())`
cell = matrix[Index.at(0), Index.at(0)] ## correctly infered as a Matrix[One, One]

## Use `Index.slice` to create a slice

two = Dim[2](2)
## the signature for slice is (offset, size), rather than (start, end)
## The size argument allows us to maintain static dimensions
sliced = matrix[Index.slice(1, two), Index.slice(3, two)] ## correctly infered as a Matrix[Two, Two]
